In [2]:
#pip install neurokit2

In [24]:
import neurokit2 as nk
import pandas as pd
import plotly.graph_objects as go
import numpy as np
from scipy.signal import find_peaks
import os

In [81]:
#RUNNING IN GOOGLE COLABORATORY

from google.colab import drive
drive.mount('/content/drive/')
%cd "/content/drive/MyDrive/coding/data"

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/coding/data


In [25]:
#RUNNING IN JUPYTER
os.chdir("C:\\Users\\rarsi\\Rutgers University\\Michelle Chen - COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Padasip_Clean_Data")
#os.chdir("C:\\Users\\katgm\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Padasip_Clean_Data")

In [26]:
def read_df (subjID):
  f_path = subjID + "_lab_filtered_cleaned.csv"
  df = pd.read_csv(f_path)
  df["Block"] = df["Block"].astype(int)
  df["SubjID"] = subjID

  return df

In [27]:
cov4 = read_df("Cov4")
cov5 = read_df("Cov5")
cov7 = read_df("Cov7")
cov8 = read_df("Cov8")
cov13 = read_df("Cov13")
cov14 = read_df("Cov14")
cov20 = read_df("Cov20")
cov19 = read_df("Cov19")
cov22 = read_df("Cov22")
cov23 = read_df("Cov23")

In [6]:
cov24=read_df("Cov24")

In [28]:
#grouping by block
def windows_by_block(df):
  df["block_index"] = df.groupby(["Block"]).cumcount()+1

  df["window_num_60"] = (df["block_index"]/3840).astype(int)
  df['window_num_60str'] = df['Block'].astype(str) + "." + df['window_num_60'].astype(str)

  df["window_num_30"] = (df["block_index"]/1920).astype(int)
  df['window_num_30str'] = df['Block'].astype(str) + "." + df['window_num_30'].astype(str)

  df["window_num_10"] = (df["block_index"]/640).astype(int)
  df['window_num_10str'] = df['Block'].astype(str) + "." + df['window_num_10'].astype(str)

  return df

In [8]:
cov24a=windows_by_block(cov24)

In [29]:
cov4a = windows_by_block(cov4)
cov5a = windows_by_block(cov5)
cov7a = windows_by_block(cov7)
cov8a = windows_by_block(cov8)
cov13a = windows_by_block(cov13)
cov14a = windows_by_block(cov14)
cov20a = windows_by_block(cov20)
cov19a = windows_by_block(cov19)
cov22a = windows_by_block(cov22)
cov23a = windows_by_block(cov23)

In [30]:
#findpeaks in scipy
#can add optional parameters to the find_peaks function to adjust for minimum height, distance and threshhold
#distance here is defined as 26, which comes out to roughly 0.4s as in Campanella_2023

def scipy_peaks(df):
  peaks = find_peaks(df["padasip"], height=0, distance=26)
  peaks_series = pd.Series(peaks[0])
  df["Peaks"] = 0
  df.loc[peaks_series, "Peaks"] = 1

  return df

In [31]:
cov4b = scipy_peaks(cov4a)
cov5b = scipy_peaks(cov5a)
cov7b = scipy_peaks(cov7a)
cov8b = scipy_peaks(cov8a)
cov13b = scipy_peaks(cov13a)
cov14b = scipy_peaks(cov14a)
cov19b = scipy_peaks(cov19a)
cov20b = scipy_peaks(cov20a)
cov22b = scipy_peaks(cov22a)
cov23b = scipy_peaks(cov23a)
#cov24b=scipy_peaks(cov24a)

In [9]:
## Excluding windows

In [32]:
#Calculate peak to peak intervals, i.e. heartbeats. Check if the interval is shorter than 500ms or longer than 1200ms.
#If it is, mark the interval for exclusion
def find_exclusions(df):
  df["timestamp"] = pd.to_datetime(df["timestamp"])

  dfc = df.drop(df[df["Peaks"] == 0].index)
  dfc["diff"] = dfc['timestamp'].diff()
  dfc['size'] = (dfc["diff"] < pd.Timedelta(milliseconds=500)) | (dfc["diff"] > pd.Timedelta(milliseconds=1200))
  percent_true_block = dfc.groupby(["Block"])['size'].mean()
  percent_true_60 = dfc.groupby(["Block", "window_num_60", "window_num_60str"])['size'].mean()
  percent_true_30 = dfc.groupby(["Block", "window_num_30", "window_num_30str"])['size'].mean()
  percent_true_10 = dfc.groupby(["Block", "window_num_10", "window_num_10str"])['size'].mean()

  exdf_Block = percent_true_block.to_frame()
  exdf_Block['exclude'] = exdf_Block["size"] > 0.2  

  exdf_60 = percent_true_60.to_frame()
  exdf_60['exclude'] = exdf_60["size"] > 0.15

  exdf_30 = percent_true_30.to_frame()
  exdf_30['exclude'] = exdf_30["size"] > 0.15

  exdf_10 = percent_true_10.to_frame()
  exdf_10['exclude'] = exdf_10["size"] > 0.15


  #descriptive stats of the windows
  SubjID = df["SubjID"][0]

  print(SubjID + "'s number of excluded blocks: " + str(exdf_Block["exclude"].sum()))  
  print(SubjID + "'s percentage of excluded intervals by block: ")
  print(exdf_Block.groupby(["Block"])['size'].mean()*100)
    
  print(SubjID + "'s percentage of Total Excluded 60s Windows: " + str(exdf_60["exclude"].mean()*100))
  print("Percentage of Excluded 60s Windows by Block: ")
  print(exdf_60.groupby(["Block"])['exclude'].mean()*100)

  print(SubjID + "'s percentage of Total Excluded 30s Windows: " + str(exdf_30["exclude"].mean()*100))
  print("Percentage of Excluded 30s Windows by Block: ")
  print(exdf_30.groupby(["Block"])['exclude'].mean()*100)

  print(SubjID + "'s percentage of Total Excluded 10s Windows: " + str(exdf_10["exclude"].mean()*100))
  print("Percentage of Excluded 10s Windows by Block: ")
  print(exdf_10.groupby(["Block"])['exclude'].mean()*100)

  return (exdf_60, exdf_30, exdf_10, exdf_Block)

In [33]:
cov4c = find_exclusions(cov4b)
cov5c = find_exclusions(cov5b)
cov7c = find_exclusions(cov7b)
cov8c = find_exclusions(cov8b)
cov13c = find_exclusions(cov13b) #exclude participant, 49-55% excluded
cov14c = find_exclusions(cov14b) #exclude?22-28% excluded
cov19c = find_exclusions(cov19b)
cov20c = find_exclusions(cov20b)
cov22c = find_exclusions(cov22b) #exclude participant 43-52% excluded
cov23c = find_exclusions(cov23b)
#cov24c = find_exclusions(cov24b) #100%????

Cov4's number of excluded blocks: 0
Cov4's percentage of excluded intervals by block: 
Block
0    2.045455
1    2.072539
2    1.583113
3    2.472527
4    2.094241
5    1.602564
6    1.892744
Name: size, dtype: float64
Cov4's percentage of Total Excluded 60s Windows: 0.0
Percentage of Excluded 60s Windows by Block: 
Block
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
Name: exclude, dtype: float64
Cov4's percentage of Total Excluded 30s Windows: 0.0
Percentage of Excluded 30s Windows by Block: 
Block
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
5    0.0
6    0.0
Name: exclude, dtype: float64
Cov4's percentage of Total Excluded 10s Windows: 1.4851485148514851
Percentage of Excluded 10s Windows by Block: 
Block
0    2.941176
1    0.000000
2    0.000000
3    3.448276
4    3.333333
5    0.000000
6    0.000000
Name: exclude, dtype: float64
Cov5's number of excluded blocks: 0
Cov5's percentage of excluded intervals by block: 
Block
0    4.909561
1    5.432099
2    4.846939
3  

In [34]:
#this function excludes both blocks and windows. Blocks are excluded if more than 20% of the windows in a given block are invalid.
# Windows are excluded if 15% of the intervals or more are invalid, as calculated in the find_exclusions function

#where df is the original dataframe i.e. cov4b, and exclusions is the dataframe of excluded blocks, i.e. cov4c
def exclude_blocks(df, exclusions):
    ex_60 = exclusions[0].reset_index()
    ex_30 = exclusions[1].reset_index()
    ex_10 = exclusions[2].reset_index()
    ex_block = exclusions[3].reset_index()
    
    #part 1: exclude on the window level (we do it this order to avoid errors; has the same outcome)
    ex_60_list = ex_60.query('exclude==True')
    df_60 = df[~df.loc[:,'window_num_60str'].isin(ex_60_list['window_num_60str'])]
    
    ex_30_list = ex_30.query('exclude==True')
    df_30 = df[~df.loc[:,'window_num_30str'].isin(ex_30_list['window_num_30str'])]
    
    ex_10_list = ex_10.query('exclude==True')
    df_10 = df[~df.loc[:,'window_num_10str'].isin(ex_10_list['window_num_10str'])]
    
    #part 2: exclude on the block level
    blocks_60 = ex_60.groupby(["Block"])['exclude'].mean()
    blocks_60 = blocks_60.reset_index(level='Block')
    blocks_60["ex_block"] = blocks_60["exclude"] > 0.2
    
    ex_blocks_60 = blocks_60.query('ex_block==True')
    df_60b = df_60[~df_60.loc[:,'Block'].isin(ex_blocks_60['Block'])]
    
    blocks_30 = ex_30.groupby(["Block"])['exclude'].mean()
    blocks_30 = blocks_30.reset_index(level='Block')
    blocks_30["ex_block"] = blocks_30["exclude"] > 0.2
    
    ex_blocks_30 = blocks_30.query('ex_block==True')
    df_30b = df_30[~df_30.loc[:,'Block'].isin(ex_blocks_30['Block'])]
    
    blocks_10 = ex_10.groupby(["Block"])['exclude'].mean()
    blocks_10 = blocks_10.reset_index(level='Block')
    blocks_10["ex_block"] = blocks_10["exclude"] > 0.2
    
    ex_blocks_10 = blocks_10.query('ex_block==True')
    df_10b = df_10[~df_10.loc[:,'Block'].isin(ex_blocks_10['Block'])]


    #create dataframes with just blocks excluded and ignore windows
    ex_blocks_blocks = ex_block.query('exclude==True')
    df_blockb = df[~df.loc[:,'Block'].isin(ex_blocks_blocks['Block'])]
    
    colsBlock = ['SubjID', 'Block', 'timestamp','padasip','Peaks','Fatigue_Rating','ACC_X_poly', 'ACC_Y_poly', 'ACC_Z_poly', 'EDA (4 Hz)','TEMP (4 Hz)']
    df_blockb = df_blockb[colsBlock]

    #exclude last block for being too small
    #for 60s block, should be 3840 obs in each window, but the first one is always 3839. use 3838 to be safe
    df_60b = df_60b.groupby(["Block", "window_num_60"]).filter(lambda x: len(x) > 3838)
    #for 30s block, should be 1920 obs in each window, first is always 1919
    df_30b = df_30b.groupby(["Block", "window_num_30"]).filter(lambda x: len(x) > 1918)
    #for 10s block, should be 640 obs in each window. first is 639 so do >638
    df_10b = df_10b.groupby(["Block", "window_num_10"]).filter(lambda x: len(x) > 638)

    cols60 = ['SubjID', 'Block', 'window_num_60', 'timestamp','padasip','Peaks','Fatigue_Rating','ACC_X_poly', 'ACC_Y_poly', 'ACC_Z_poly', 'EDA (4 Hz)','TEMP (4 Hz)']
    df_60b = df_60b[cols60]

    cols30 = ['SubjID', 'Block', 'window_num_30', 'timestamp','padasip','Peaks','Fatigue_Rating','ACC_X_poly', 'ACC_Y_poly', 'ACC_Z_poly', 'EDA (4 Hz)','TEMP (4 Hz)']
    df_30b = df_30b[cols30]
    
    cols10 = ['SubjID', 'Block', 'window_num_10', 'timestamp','padasip','Peaks','Fatigue_Rating','ACC_X_poly', 'ACC_Y_poly', 'ACC_Z_poly', 'EDA (4 Hz)','TEMP (4 Hz)']
    df_10b = df_10b[cols10]

    
    return (df_60b, df_30b, df_10b, df_blockb)

In [35]:
cov4d = exclude_blocks(cov4b, cov4c)
cov5d = exclude_blocks(cov5b, cov5c)
cov7d = exclude_blocks(cov7b, cov7c)
cov8d = exclude_blocks(cov8b, cov8c)
cov13d = exclude_blocks(cov13b, cov13c)
cov14d = exclude_blocks(cov14b, cov14c)
cov19d = exclude_blocks(cov19b, cov19c)
cov20d = exclude_blocks(cov20b, cov20c)
cov22d = exclude_blocks(cov22b, cov22c)
cov23d = exclude_blocks(cov23b, cov23c)
#cov24d = exclude_blocks(cov24b, cov24c)

In [36]:
cov23d[0]

,SubjID,Block,window_num_60,timestamp,padasip,Peaks,Fatigue_Rating,ACC_X_poly,ACC_Y_poly,ACC_Z_poly,EDA (4 Hz),TEMP (4 Hz)
0,Cov23,0,0,2023-08-24 09:44:56.734375,-2.382651,0,37.0,-6.196748,-5.386565,62.131215,NaN,NaN
1,Cov23,0,0,2023-08-24 09:44:56.750000,-1.992215,0,37.0,-6.368756,-5.524431,61.860389,0.024328,28.81
2,Cov23,0,0,2023-08-24 09:44:56.765625,-1.456650,0,37.0,-6.597882,-5.561790,61.774761,NaN,NaN
3,Cov23,0,0,2023-08-24 09:44:56.781250,-0.787989,0,37.0,-6.882530,-5.546614,61.835790,NaN,NaN
4,Cov23,0,0,2023-08-24 09:44:56.796875,-0.007793,0,37.0,-7.208435,-5.533380,61.986558,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
119438,Cov23,6,2,2023-08-24 10:20:23.687500,1.676715,0,58.0,33.297352,10.572207,70.624727,NaN,NaN
119439,Cov23,6,2,2023-08-24 10:20:23.703125,1.719970,0,58.0,27.019728,9.577152,65.131536,NaN,NaN
119440,Cov23,6,2,2023-08-24 10:20:23.718750,1.727937,1,58.0,22.367685,8.618348,59.947219,NaN,NaN
119441,Cov23,6,2,2023-08-24 10:20:23.734375,1.692459,0,58.0,19.886606,7.742974,55.546803,NaN,NaN


In [37]:
#os.chdir("C:\\Users\\katgm\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Windowed_Data")
os.chdir("C:\\Users\\rarsi\\Rutgers University\\Michelle Chen - COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Windowed_Data")

cov4d[0].to_csv("Cov4_60s.csv")
cov4d[1].to_csv("Cov4_30s.csv")
cov4d[2].to_csv("Cov4_10s.csv")

cov5d[0].to_csv("Cov5_60s.csv")
cov5d[1].to_csv("Cov5_30s.csv")
cov5d[2].to_csv("Cov5_10s.csv")

cov7d[0].to_csv("Cov7_60s.csv")
cov7d[1].to_csv("Cov7_30s.csv")
cov7d[2].to_csv("Cov7_10s.csv")

cov8d[0].to_csv("Cov8_60s.csv")
cov8d[1].to_csv("Cov8_30s.csv")
cov8d[2].to_csv("Cov8_10s.csv")

cov13d[0].to_csv("Cov13_60s.csv")
cov13d[1].to_csv("Cov13_30s.csv")
cov13d[2].to_csv("Cov13_10s.csv")

cov14d[0].to_csv("Cov14_60s.csv")
cov14d[1].to_csv("Cov14_30s.csv")
cov14d[2].to_csv("Cov14_10s.csv")

cov19d[0].to_csv("Cov19_60s.csv")
cov19d[1].to_csv("Cov19_30s.csv")
cov19d[2].to_csv("Cov19_10s.csv")

cov20d[0].to_csv("Cov20_60s.csv")
cov20d[1].to_csv("Cov20_30s.csv")
cov20d[2].to_csv("Cov20_10s.csv")

cov22d[0].to_csv("Cov22_60s.csv")
cov22d[1].to_csv("Cov22_30s.csv")
cov22d[2].to_csv("Cov22_10s.csv")

cov23d[0].to_csv("Cov23_60s.csv")
cov23d[1].to_csv("Cov23_30s.csv")
cov23d[2].to_csv("Cov23_10s.csv")

# cov24d[0].to_csv("Cov24_60s.csv")
# cov24d[1].to_csv("Cov24_30s.csv")
# cov24d[2].to_csv("Cov24_10s.csv")


In [38]:
#os.chdir("C:\\Users\\katgm\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Windowed_Data")
os.chdir("C:\\Users\\rarsi\\Rutgers University\\Michelle Chen - COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\Windowed_Data")

cov4d[3].to_csv("Cov4_block.csv")
cov5d[3].to_csv("Cov5_block.csv")
cov7d[3].to_csv("Cov7_block.csv")
cov8d[3].to_csv("Cov8_block.csv")
cov13d[3].to_csv("Cov13_block.csv")
cov14d[3].to_csv("Cov14_block.csv")
cov19d[3].to_csv("Cov19_block.csv")
cov20d[3].to_csv("Cov20_block.csv")
cov22d[3].to_csv("Cov22_block.csv")
cov23d[3].to_csv("Cov23_block.csv")
#cov24d[3].to_csv("Cov24_block.csv")

In [17]:
def get_features(x):
  return nk.hrv(x, sampling_rate=64)

In [18]:
#grouping by block
def features_by_windows(df, window_size):
  by_block = df.groupby(["Block", window_size, "SubjID", "Fatigue_Rating"])['Peaks']
  df = by_block.apply(get_features)
  return df

In [19]:
cov4e_60 = features_by_windows(cov4d[0], "window_num_60")
cov4e_30 = features_by_windows(cov4d[1], "window_num_30")
#cov4e_10 = features_by_windows(cov4d[2], "window_num_10")

cov5e_60 = features_by_windows(cov5d[0], "window_num_60")
cov5e_30 = features_by_windows(cov5d[1], "window_num_30")
#cov5e_10 = features_by_windows(cov5d[2], "window_num_10")

cov7e_60 = features_by_windows(cov7d[0], "window_num_60")
cov7e_30 = features_by_windows(cov7d[1], "window_num_30")
#cov7e_10 = features_by_windows(cov7d[2], "window_num_10")

cov8e_60 = features_by_windows(cov8d[0], "window_num_60")
cov8e_30 = features_by_windows(cov8d[1], "window_num_30")
#cov8e_10 = features_by_windows(cov8d[2], "window_num_10")

cov13e_60 = features_by_windows(cov13d[0], "window_num_60")
cov13e_30 = features_by_windows(cov13d[1], "window_num_30")
#cov13e_10 = features_by_windows(cov13d[2], "window_num_10")

cov14e_60 = features_by_windows(cov14d[0], "window_num_60")
cov14e_30 = features_by_windows(cov14d[1], "window_num_30")
#cov14e_10 = features_by_windows(cov14d[2], "window_num_10")

cov19e_60 = features_by_windows(cov19d[0], "window_num_60")
cov19e_30 = features_by_windows(cov19d[1], "window_num_30")
#cov19e_10 = features_by_windows(cov19d[2], "window_num_10")

cov20e_60 = features_by_windows(cov20d[0], "window_num_60")
cov20e_30 = features_by_windows(cov20d[1], "window_num_30")
#cov20e_10 = features_by_windows(cov20d[2], "window_num_10")

cov22e_60 = features_by_windows(cov22d[0], "window_num_60")
cov22e_30 = features_by_windows(cov22d[1], "window_num_30")
#cov22e_10 = features_by_windows(cov22d[2], "window_num_10")

cov23e_60 = features_by_windows(cov23d[0], "window_num_60")
cov23e_30 = features_by_windows(cov23d[1], "window_num_30")
#cov23e_10 = features_by_windows(cov23d[2], "window_num_10")

C:\Users\rarsi\AppData\Local\Programs\Python\Python312\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\rarsi\AppData\Local\Programs\Python\Python312\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long-term correlation is smaller than the minimum duration of windows. Refer to the `scale` argument in `nk.fractal_dfa()` for more information.
  warn(
C:\Users\rarsi\AppData\Local\Programs\Python\Python312\Lib\site-packages\neurokit2\hrv\hrv_nonlinear.py:474: NeuroKitWarning: DFA_alpha2 related indices will not be calculated. The maximum duration of the windows provided for the long

In [20]:
def features_by_block(df):
  by_block = df.groupby(["Block", "SubjID", "Fatigue_Rating"])['Peaks']
  df = by_block.apply(get_features)
  return df

cov4e_block = features_by_block(cov4d[3])
cov5e_block = features_by_block(cov5d[3])
cov7e_block = features_by_block(cov7d[3])
cov8e_block = features_by_block(cov8d[3])
cov13e_block = features_by_block(cov13d[3])
cov14e_block = features_by_block(cov14d[3])
cov19e_block = features_by_block(cov19d[3])
cov20e_block = features_by_block(cov20d[3])
cov22e_block = features_by_block(cov22d[3])
cov23e_block = features_by_block(cov23d[3])
#cov24e_block = features_by_block(cov24d[3])

In [21]:
#os.chdir("C:\\Users\\katgm\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\HRV_features")
os.chdir("C:\\Users\\rarsi\\Rutgers University\\Michelle Chen - COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\HRV_features")

cov4e_60.to_csv("Cov4_features60.csv")
cov4e_30.to_csv("Cov4_features30.csv")
#cov4e_10.to_csv("Cov4_features10.csv")

cov5e_60.to_csv("Cov5_features60.csv")
cov5e_30.to_csv("Cov5_features30.csv")
#cov5e_10.to_csv("Cov5_features10.csv")

cov7e_60.to_csv("Cov7_features60.csv")
cov7e_30.to_csv("Cov7_features30.csv")
#cov7e_10.to_csv("Cov7_features10.csv")

cov8e_60.to_csv("Cov8_features60.csv")
cov8e_30.to_csv("Cov8_features30.csv")
#cov8e_10.to_csv("Cov8_features10.csv")

cov13e_60.to_csv("Cov13_features60.csv")
cov13e_30.to_csv("Cov13_features30.csv")
#cov13e_10.to_csv("Cov13_features10.csv")

cov14e_60.to_csv("Cov14_features60.csv")
cov14e_30.to_csv("Cov14_features30.csv")
#cov14e_10.to_csv("Cov14_features10.csv")

cov19e_60.to_csv("Cov19_features60.csv")
cov19e_30.to_csv("Cov19_features30.csv")
#cov19e_10.to_csv("Cov19_features10.csv")

cov20e_60.to_csv("Cov20_features60.csv")
cov20e_30.to_csv("Cov20_features30.csv")
#cov20e_10.to_csv("Cov20_features10.csv")

cov22e_60.to_csv("Cov22_features60.csv")
cov22e_30.to_csv("Cov22_features30.csv")
#cov22e_10.to_csv("Cov22_features10.csv")

cov23e_60.to_csv("Cov23_features60.csv")
cov23e_30.to_csv("Cov23_features30.csv")
#cov23e_10.to_csv("Cov23_features10.csv")

In [23]:
#os.chdir("C:\\Users\\katgm\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\HRV_features")
os.chdir("C:\\Users\\rarsi\\Rutgers University\\Michelle Chen - COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\HRV_features")
cov4e_block.to_csv("Cov4_block_features.csv")
cov5e_block.to_csv("Cov5_block_features.csv")
cov7e_block.to_csv("Cov7_block_features.csv")
cov8e_block.to_csv("Cov8_block_features.csv")
cov13e_block.to_csv("Cov13_block_features.csv")
cov14e_block.to_csv("Cov14_block_features.csv")
cov19e_block.to_csv("Cov19_block_features.csv")
cov20e_block.to_csv("Cov20_block_features.csv")
cov22e_block.to_csv("Cov22_block_features.csv")
cov23e_block.to_csv("Cov23_block_features.csv")
#cov24e_block.to_csv("Cov24_block_features.csv")